In [1]:
#第5章準備 実行は一度で良い(実行済み)

import zipfile

# zipファイルに含まれているファイルの取り出し
with zipfile.ZipFile("./data/ai.ja.zip") as myzip:
    myzip.extractall()

!cabocha -f1 -o ai.ja.txt.parsed ai.ja.txt 


In [2]:
#40. 係り受け解析結果の読み込み(形態素)

class Morph:
    def __init__(self, morph):
        surface, attribute = morph.split("\t")
        attribute = attribute.split(",")
        
        self.surface = surface
        self.base = attribute[6]
        self.pos = attribute[0]
        self.pos1 = attribute[1]
        
File = "./data/ai.ja.txt.parsed"

sentences = []
morphs = []

with open(File, mode="r") as f:
    for line in f:
        if line[0] == "*":
            continue
        elif line != "EOS\n":
            morphs.append(Morph(line))
        else:
            sentences.append(morphs)
            morphs = []

for i in sentences[4]:
    print(vars(i))

{'surface': '『', 'base': '『', 'pos': '記号', 'pos1': '括弧開'}
{'surface': '日本', 'base': '日本', 'pos': '名詞', 'pos1': '固有名詞'}
{'surface': '大', 'base': '大', 'pos': '接頭詞', 'pos1': '名詞接続'}
{'surface': '百科全書', 'base': '百科全書', 'pos': '名詞', 'pos1': '一般'}
{'surface': '(', 'base': '*\n', 'pos': '名詞', 'pos1': 'サ変接続'}
{'surface': 'ニッポニカ', 'base': '*\n', 'pos': '名詞', 'pos1': '一般'}
{'surface': ')』', 'base': '*\n', 'pos': '名詞', 'pos1': 'サ変接続'}
{'surface': 'の', 'base': 'の', 'pos': '助詞', 'pos1': '連体化'}
{'surface': '解説', 'base': '解説', 'pos': '名詞', 'pos1': 'サ変接続'}
{'surface': 'で', 'base': 'で', 'pos': '助詞', 'pos1': '格助詞'}
{'surface': '、', 'base': '、', 'pos': '記号', 'pos1': '読点'}
{'surface': '情報', 'base': '情報', 'pos': '名詞', 'pos1': '一般'}
{'surface': '工学', 'base': '工学', 'pos': '名詞', 'pos1': '一般'}
{'surface': '者', 'base': '者', 'pos': '名詞', 'pos1': '接尾'}
{'surface': '・', 'base': '・', 'pos': '記号', 'pos1': '一般'}
{'surface': '通信', 'base': '通信', 'pos': '名詞', 'pos1': 'サ変接続'}
{'surface': '工学', 'base': '工学', 'pos': '名詞', 

In [3]:
#41. 係り受け解析結果の読み込み(文節・係り受け)

class Morph:
    def __init__(self, line):
        surface, attribute = line.split("\t")
        attribute = attribute.split(",")
        
        self.surface = surface
        self.base = attribute[6]
        self.pos = attribute[0]
        self.pos1 = attribute[1]

class Chunk:
    def __init__(self, morphs, dst):
        self.morphs = morphs
        self.dst = dst
        self.srcs = []
        
    def __str__(self):
        return ''.join( mor.surface for mor in morphs )
        
class Sentence:
    def __init__(self, chunks):
        self.chunks = chunks
        for i, chunk in enumerate(self.chunks): 
            if chunk.dst != -1:
                self.chunks[chunk.dst].srcs.append(i)
        
File = "./data/ai.ja.txt.parsed"


morphs = []
chunks = []
sentences = []

with open(File, mode="r") as f:
    for line in f:
        if line[0] == "*":
            if len(morphs) > 0:
                chunks.append(Chunk(morphs, dst))
                #print( str(Chunk(morphs, dst)) )
                morphs = []
            dst = int(line.split(" ")[2].rstrip("D"))
        elif line != "EOS\n":
            morphs.append(Morph(line))
        else:
            chunks.append(Chunk(morphs, dst))
            sentences.append(Sentence(chunks))
            morphs = []
            chunks = []
            dst = 0
            
for chunk in sentences[2].chunks:
    print([morph.surface for morph in chunk.morphs], chunk.dst, chunk.srcs)

['人工', '知能'] 17 []
['（', 'じん', 'こうち', 'のう', '、', '、'] 17 []
['AI'] 3 []
['〈', 'エーアイ', '〉', '）', 'と', 'は', '、'] 17 [2]
['「', '『', '計算'] 5 []
['（', '）', '』', 'という'] 9 [4]
['概念', 'と'] 9 []
['『', 'コンピュータ'] 8 []
['（', '）', '』', 'という'] 9 [7]
['道具', 'を'] 10 [5, 6, 8]
['用い', 'て'] 12 [9]
['『', '知能', '』', 'を'] 12 []
['研究', 'する'] 13 [10, 11]
['計算', '機', '科学'] 14 [12]
['（', '）', 'の'] 15 [13]
['一', '分野', '」', 'を'] 16 [14]
['指す'] 17 [15]
['語', '。'] 34 [0, 1, 3, 16]
['「', '言語', 'の'] 20 []
['理解', 'や'] 20 []
['推論', '、'] 21 [18, 19]
['問題', '解決', 'など', 'の'] 22 [20]
['知的', '行動', 'を'] 24 [21]
['人間', 'に'] 24 []
['代わっ', 'て'] 26 [22, 23]
['コンピューター', 'に'] 26 []
['行わ', 'せる'] 27 [24, 25]
['技術', '」', '、', 'または', '、'] 34 [26]
['「', '計算', '機'] 29 []
['（', 'コンピュータ', '）', 'による'] 31 [28]
['知的', 'な'] 31 []
['情報処理', 'システム', 'の'] 33 [29, 30]
['設計', 'や'] 33 []
['実現', 'に関する'] 34 [31, 32]
['研究', '分野', '」', 'と', 'も'] 35 [17, 27, 33]
['さ', 'れる', '。'] -1 [34]
